In [2]:
import cv2
import os

def extract_frames(video_path, output_folder):
    """
    Extracts frames from a video at variable rates based on the timestamp.
    - 10 FPS for the beginning and end sections.
    - 2 FPS for the middle section (50s to 3m 50s).

    Args:
        video_path (str): The full path to the input video file.
        output_folder (str): The path to the folder where frames will be saved.
    """
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        print(f"Created directory: {output_folder}")

    # Open the video file
    video_capture = cv2.VideoCapture(video_path)
    if not video_capture.isOpened():
        print(f"Error: Could not open video file at {video_path}")
        return

    # --- NEW LOGIC FOR VARIABLE FRAME RATE ---
    
    # 1. Define time intervals and frame rates in seconds
    slow_rate_start_sec = 50
    slow_rate_end_sec = (3 * 60) + 50  # 3 minutes and 50 seconds is 230 seconds

    # 2. Calculate the desired time interval between saved frames for each rate
    interval_fast_rate = 1.0 / 10  # For 10 frames per second
    interval_slow_rate = 1.0 / 2   # For 2 frames per second
    
    # 3. Variable to track the timestamp of the last saved frame
    last_saved_timestamp_sec = -1  # Initialize to ensure the first frame is considered

    print("Starting frame extraction with variable rates...")
    saved_frame_count = 0
    
    while True:
        success, frame = video_capture.read()
        if not success:
            break

        # Get the current timestamp of the frame in milliseconds and convert to seconds
        current_timestamp_sec = video_capture.get(cv2.CAP_PROP_POS_MSEC) / 1000.0

        # Determine which frame rate interval we should be using
        if slow_rate_start_sec <= current_timestamp_sec <= slow_rate_end_sec:
            # We are in the 2 FPS zone
            target_interval = interval_slow_rate
        else:
            # We are in the 10 FPS zone
            target_interval = interval_fast_rate

        # Check if enough time has passed since the last frame was saved
        if current_timestamp_sec >= last_saved_timestamp_sec + target_interval:
            # Construct the output file name
            frame_filename = os.path.join(output_folder, f"frame_{saved_frame_count}.jpg")
            
            # Save the current frame
            cv2.imwrite(frame_filename, frame)
            
            # Update the last saved timestamp and count
            last_saved_timestamp_sec = current_timestamp_sec
            saved_frame_count += 1

    # Release the video capture object
    video_capture.release()
    
    print(f"✅ Successfully extracted {saved_frame_count} frames to '{output_folder}'")

In [3]:
# 1. Set the path to your video file
input_video_file = "primary.avi"  # <-- CHANGE THIS to your video file name

# 2. Set the name for the output folder
output_frames_folder = "extracted_frames"

# 3. Call the function to start the extraction!
extract_frames(input_video_file, output_frames_folder)

Created directory: extracted_frames
Starting frame extraction with variable rates...
✅ Successfully extracted 1258 frames to 'extracted_frames'
